# External Connection Configuration

> **Cel:** Konfiguracja połączenia Unity Catalog z zewnętrznym storage (Azure Data Lake Storage Gen2) za pomocą Access Connector i External Location.

**Wymagania wstępne:**
- Azure Databricks Workspace z Unity Catalog
- Azure Data Lake Storage Gen2 (ADLS)
- Uprawnienia administratora w Azure Portal i Databricks Workspace

---

### Spis treści

| Krok | Opis |
|------|------|
| **Część 1** | Pobranie Resource ID Access Connectora |
| **Część 2** | Nadanie uprawnień RBAC do Storage Account |
| **Część 3** | Utworzenie Storage Credential w Databricks |
| **Część 4** | Utworzenie External Location |
| **Część 5** | Konfiguracja uprawnień dostępu |

## Część 1: Pobranie Resource ID Access Connectora

Przechodzimy do **Azure Portal** → nasz Workspace → **Managed Resource Group**.

![Managed Resource Group](../assets/images/training_2026/cheatsheet/17cf776d15ce4ffcb87d64d7b28d6107.webp)

Wewnątrz Managed Resource Group znajdujemy zasób **Access Connector for Azure Databricks** (`unity-access-connector`). Kopiujemy jego **Resource ID** — będzie potrzebny w kolejnych krokach.

![Access Connector Resource ID](../assets/images/training_2026/cheatsheet/952e125cc24944499947d4e95d089cc4.webp)

> **Uwaga:** Resource ID ma format:  
> `/subscriptions/<sub-id>/resourceGroups/<rg>/providers/Microsoft.Databricks/accessConnectors/<name>`

## Część 2: Nadanie uprawnień RBAC na Storage Account

Przechodzimy do naszego **Azure Data Lake Storage Gen2** → **Access Control (IAM)**.

![Storage Account IAM](../assets/images/training_2026/cheatsheet/a67bf2bdb6cb4a2cb6b30897a7182b2b.webp)

Dodajemy rolę stosując **zasadę najmniejszych uprawnień** (Principle of Least Privilege):

| Rola | Uprawnienia | Zastosowanie |
|------|-------------|-------------|
| **Storage Blob Data Reader** | Tylko odczyt | Produkcja (read-only) |
| **Storage Blob Data Contributor** | Odczyt + zapis | Szkolenie / development |

> **Na potrzeby szkolenia** nadajemy rolę **Storage Blob Data Contributor**.  
> W środowisku produkcyjnym stosuj minimalny wymagany zakres uprawnień.

![Role Assignment](../assets/images/training_2026/cheatsheet/fdcf7b03b80e44c499a0f88147a69361.webp)

Przechodzimy do zakładki **Members** → **Select members** → wybieramy typ **Managed Identity** → filtrujemy po **Access Connector for Azure Databricks** i wybieramy nasz connector.

![Member Selection - Managed Identity](../assets/images/training_2026/cheatsheet/36092c21a0034e12a87e2b5e2f624501.webp)

Klikamy **Review + Assign** — gotowe. Access Connector ma teraz uprawnienia do Storage Account.

## Część 3: Utworzenie Storage Credential w Databricks

Przechodzimy do **Databricks Workspace** → **Catalog** → **External Data** → **Credentials**.

![Catalog - External Data](../assets/images/training_2026/cheatsheet/d369852738144865a154b4e46bebe71a.webp)

Tworzymy nowy credential (jeśli jeszcze nie istnieje):

| Pole | Wartość |
|------|--------|
| **Credential name** | Wybrana nazwa (np. `adls_training_credential`) |
| **Access connector ID** | Resource ID skopiowany w Części 1 |

> **Tip:** Credential jest współdzielony — można go używać w wielu External Locations.

## Część 4: Utworzenie External Location

Wracamy do zakładki **External Locations** → klikamy **Create External Location**.

Wypełniamy formularz:

| Pole | Wartość | Przykład |
|------|---------|--------|
| **External location name** | Nazwa własna | `training_landing_zone` |
| **URL** | Ścieżka ABFSS do kontenera | `abfss://container@account.dfs.core.windows.net/path` |
| **Storage credential** | Credential utworzony w Części 3 | `adls_training_credential` |

> **Ważne:** Przed utworzeniem External Location warto wcześniej stworzyć dedykowany kontener na ADLS pod dane rozwiązanie.

![External Location - formularz](../assets/images/training_2026/cheatsheet/e037cfc8cc554ee491f9367f3101b49b.webp)

Klikamy **Create** i czekamy na walidację połączenia. Jeśli widzimy poniższy output — konfiguracja zakończona sukcesem:

![External Location - sukces](../assets/images/training_2026/cheatsheet/a1373fa1fe0741699a3d29fd38868640.webp)

## Część 5: Konfiguracja uprawnień dostępu

Ostatni krok — w zakładce **Permissions** External Location nadajemy odpowiednie uprawnienia użytkownikom/grupom.

| Uprawnienie | Opis |
|-------------|------|
| `READ FILES` | Odczyt plików z lokalizacji |
| `WRITE FILES` | Zapis plików do lokalizacji |
| `CREATE EXTERNAL TABLE` | Tworzenie tabel zewnętrznych |
| `ALL PRIVILEGES` | Pełny dostęp (tylko dev/demo!) |

> **⚠️ UWAGA:** Na potrzeby szkolenia nadano `ALL PRIVILEGES`.  
> **Na produkcji nigdy tego nie rób!** Stosuj granularne uprawnienia zgodnie z polityką governance Twojej organizacji.

![Permissions](../assets/images/training_2026/cheatsheet/419e8741a5304838931458986d191083.webp)

---

### Podsumowanie

```
Azure Portal                          Databricks Workspace
─────────────                         ────────────────────
1. Managed Resource Group             3. Catalog → Credentials
   └─ Access Connector (Resource ID)     └─ Storage Credential (name + Resource ID)
                                      
2. Storage Account → IAM              4. Catalog → External Locations
   └─ Role Assignment                    └─ External Location (URL + Credential)
     (Managed Identity + RBAC Role)   
                                      5. External Location → Permissions
                                         └─ Granularne uprawnienia
```